In [4]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True



In [6]:
"""
train_nlow = np.zeros((df['Page content'].values.shape[0], 4), dtype=object)
test_nlow = np.zeros((df_test['Page content'].values.shape[0], 4), dtype=object)

for i in range(df['Page content'].values.shape[0]):
    title, text, author, nf = procd.parse_html(df['Page content'][i], tolower=False)
    train_nlow[i, 0] = df['Id'][i]
    train_nlow[i, 1] = df['Popularity'][i]
    train_nlow[i, 2] = title
    train_nlow[i, 3] = text

for i in range(df_test['Page content'].values.shape[0]):
    title, text, author, nf = procd.parse_html(df_test['Page content'][i], tolower=False)
    test_nlow[i, 0] = df['Id'][i]
    test_nlow[i, 1] = df['Popularity'][i]
    test_nlow[i, 2] = title
    test_nlow[i, 3] = text

pd.DataFrame(data=train_nlow, columns=['Id', 'Popularity', 'title', 'content']).to_csv('train_parse_ncase.csv', index=False)
pd.DataFrame(data=test_nlow, columns=['Id', 'Popularity', 'title', 'content']).to_csv('test_parse_ncase.csv', index=False)
"""

In [1]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /home/cgv03/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
"""
train_channel = np.zeros((df.values.shape[0], 4), dtype=object)
test_channel = np.zeros((df_test.values.shape[0], 4), dtype=object)
columns = ['entertainment', 'world', 'social-media', 'tech']


for i in range(df['Page content'].values.shape[0]):
    channel = procd.parse_channel(df['Page content'][i])
    if channel in columns:
        train_channel[i, columns.index(channel)] = 1

pd.DataFrame(data=train_channel, columns=columns).to_csv('train_channel.csv', index=False)
        
for i in range(df_test['Page content'].values.shape[0]):
    channel = procd.parse_channel(df_test['Page content'][i])
    if channel in columns:
        test_channel[i, columns.index(channel)] = 1
        
pd.DataFrame(data=test_channel, columns=columns).to_csv('test_channel.csv', index=False)
"""
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
#for i in range(5):

    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

#for i in range(5):

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')
"""

"\ntrain_parse = np.zeros((df.values.shape[0], 5),dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n#for i in range(5):\n\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]            = procd.parse_html(df['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')\n\ntrain_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)\n\n#for i in range(5):\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]             = procd.parse_html(df_test['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')\n"

In [23]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')
print(df['author'][10])

AmandaWills


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
n_topics = 10
n_top_words = 15
n_features = 2**20
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                tokenizer=tokenizer_stem_nostop)
tf = tf_vectorizer.fit_transform(df['title'])

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
topic_tf = lda.fit_transform(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0:
game video show twitter tv microsoft new user get play song robot deal throne final
Topic #1:
launch turn take fan vine still cat plan twitter travel record everi hous bill network
Topic #2:
appl iphon app come watch samsung new thing phone amazon android ipad free io best
Topic #3:
year top week man old next music dead announc dog video stop made right christma
Topic #4:
googl know need u one news want mobil glass kid obama give app design data
Topic #5:
new time life make internet trailer york movi real break bring develop add updat offic
Topic #6:
media way social tech job chang startup busi protest love open market see american high
Topic #7:
world facebook ad like report use say get look million may digit cup app appl
Topic #8:
photo star day onlin tweet could war space power kill celebr season new win ever
Topic #9:
help smartphon peopl youtub live home hit instagram creat work billion pro watch back put



In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfdoc = TfidfVectorizer(max_features=2**20, tokenizer=tokenizer_stem_nostop)
tfdoc.fit(df['content'])
doc = tfdoc.transform(df['content'])

tfaut = TfidfVectorizer(max_features=2**10)
author = tfaut.fit_transform(df['author'])


In [24]:
from nltk.tree import Tree
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
"""
tag_types = ['FACILITY', 'GPE', 'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']

df_ncase = pd.read_csv('train_parse_ncase.csv')['content'].values
dt_ncase = pd.read_csv('test_parse_ncase.csv')['content'].values

train_ner = np.zeros((df_ncase.shape[0], 6 * 2), dtype=int)
test_ner  = np.zeros((dt_ncase.shape[0], 6 * 2), dtype=int)

for i in range(df_ncase.shape[0]):
    tokens = nltk.word_tokenize(df_ncase[i])
    tags = nltk.pos_tag(tokens)
    chunks = nltk.ne_chunk(tags)
    if i % 500 == 0:
        print('analyzing %d' %(i))
    for x in chunks:
        if isinstance(x, Tree):
            label = x.label()
            idx = tag_types.index(label)
            train_ner[i, idx] = 1
            train_ner[i, idx+6] += 1

pd.DataFrame(data=train_ner, columns=['has_FACILITY', 'has_GPE', 'has_GSP', 'has_LOCATION', 
                                      'has_ORGANIZATION', 'has_PERSON','FACILITY', 'GPE', 
                                      'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']).to_csv('train_ner.csv',index=False)
            
for i in range(dt_ncase.shape[0]):
    tokens = nltk.word_tokenize(dt_ncase[i])
    tags = nltk.pos_tag(tokens)
    chunks = nltk.ne_chunk(tags)
    for x in chunks:
        if isinstance(x, Tree):
            label = x.label()
            idx = tag_types.index(label)
            test_ner[i, idx] = 1
            test_ner[i, idx+6] += 1
pd.DataFrame(data=test_ner, columns=['has_FACILITY', 'has_GPE', 'has_GSP', 'has_LOCATION', 
                                      'has_ORGANIZATION', 'has_PERSON','FACILITY', 'GPE', 
                                      'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']).to_csv('test_ner.csv',index=False)
"""

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cgv03/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/cgv03/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/cgv03/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/cgv03/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"\ntag_types = ['FACILITY', 'GPE', 'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']\n\ndf_ncase = pd.read_csv('train_parse_ncase.csv')['content'].values\ndt_ncase = pd.read_csv('test_parse_ncase.csv')['content'].values\n\ntrain_ner = np.zeros((df_ncase.shape[0], 6 * 2), dtype=int)\ntest_ner  = np.zeros((dt_ncase.shape[0], 6 * 2), dtype=int)\n\nfor i in range(df_ncase.shape[0]):\n    tokens = nltk.word_tokenize(df_ncase[i])\n    tags = nltk.pos_tag(tokens)\n    chunks = nltk.ne_chunk(tags)\n    if i % 500 == 0:\n        print('analyzing %d' %(i))\n    for x in chunks:\n        if isinstance(x, Tree):\n            label = x.label()\n            idx = tag_types.index(label)\n            train_ner[i, idx] = 1\n            train_ner[i, idx+6] += 1\n\npd.DataFrame(data=train_ner, columns=['has_FACILITY', 'has_GPE', 'has_GSP', 'has_LOCATION', \n                                      'has_ORGANIZATION', 'has_PERSON','FACILITY', 'GPE', \n                                      'GSP', 'LOCATION', 'ORGA

In [244]:
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=150)
doc_x = ch2.fit_transform(doc, y_train)

cha2 = SelectKBest(chi2, k=10)
aut_x = cha2.fit_transform(author, y_train)

In [63]:
p_data = df['Popularity'] > 0
n_data = df['Popularity'] < 0

In [27]:
n_twords = np.zeros((df['title'].shape[0], 1), dtype=int)
n_twords_test = np.zeros((df_test['title'].shape[0], 1), dtype=int)
for i in range(n_twords.shape[0]):
    n_twords[i] = len(tokenizer_stem_nostop(df['title'][i]))
for i in range(n_twords_test.shape[0]):
    n_twords_test[i] = len(tokenizer_stem_nostop(df_test['title'][i]))

In [28]:
def mix(a, b):
    r = a.shape[1]
    k = b.shape[1]
    temp = None
    for i in range(r):
        for j in range(k):
            if temp: temp = a[:, i].reshape(-1,1) * b[:,j].reshape(-1,1)
            else: temp = np.concatenate((temp, a[:, i].reshape(-1,1) * b[:,j].reshape(-1,1)), axis=1)
    
    return temp

In [290]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

poly_stdn = PolynomialFeatures(degree=2)
poly_cont = PolynomialFeatures(degree=2)
poly_mixd = PolynomialFeatures(degree=2)
train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
train_channel = pd.read_csv('train_channel.csv').values
train_polarity = pd.read_csv('train_polarity.csv').values
train_links = pd.read_csv('train_links.csv').values
train_keypop = pd.read_csv('train_keypop.csv').values
train_ner = pd.read_csv('train_ner.csv').values
train_ner = train_ner[:, 6:]
train_nwords = df['n_words'].values.reshape(-1,1)
train_nfig = df['n_fig'].values.reshape(-1,1)

time_feats = np.concatenate((train_time, train_weekday), axis=1)
cont_feats = np.concatenate((train_polarity, topic_tf, train_channel), axis=1)
stdn_feats = np.concatenate((n_twords, train_nwords, train_links, train_keypop, train_ner), axis=1)
#mixd_feats = np.concatenate((train_time * tra, train_keypop, train_polarity), axis=1)
stdn_feats = sc.fit_transform(stdn_feats)
stdn_feats = poly_stdn.fit_transform(stdn_feats)[:,1:]
cont_feats = poly_cont.fit_transform(cont_feats)[:,1:]

X_train_aug = np.concatenate((time_feats, cont_feats, stdn_feats, doc_x.todense()), axis=1)

#X_train_aug = sc.fit_transform(X_train_aug)

In [291]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)

print(X_train_aug.shape)

(27643, 424)


In [292]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [316]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.neural_network import MLPClassifier
#param_c = { 'C': [0.001, 0.005, 0.01, 0.03, 0.05, 0.07, 0.1, 0.25, 0.5, 0.7, 1.0, 3.0, 10.0, 100.0, 200.0, 1000, 1000.0]}
#grid_search = GridSearchCV(LogisticRegression(random_state=0), param_grid=param_c)
#start = time()
#grid_search.fit(X_train_aug, y_train)

#print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
#      % (time() - start, len(grid_search.cv_results_['params'])))
#report(grid_search.cv_results_)


candidates = []

# former best 0.06
lr = LogisticRegression(C=0.06, random_state=0)
#lr = MLPClassifier()

#lr = GaussianNB()

lr.fit(X_htrain, y_htrain)

pred = lr.predict(X_htrain)
pred_t = lr.predict(X_htest)
score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)
print("LR score (train): %f, (test): %f"%(score_train, score_test))


score = cross_val_score(lr, X_train_aug, y_train, cv=10)
# best case here
print("LR score CV: %f"%(score.mean()))


lr.fit(X_train_aug, y_train)

pred = lr.predict(X_train_aug)
pred_t = lr.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("LR score (train): %f, (test): %f"%(score_train, score_test))
candidates.append({'clf':lr, 'C':0.06, 'score':0.557699})
"""
bag = BaggingClassifier(base_estimator=lr, n_estimators=100, max_samples=0.5, max_features=0.5,
                        oob_score=True, random_state=0, n_jobs=3)

score = cross_val_score(bag, X_train_aug, y_train, cv=10)


# best case here
print("voter score (CV): %f"%(score.mean()))
bag.fit(X_train_aug, y_train)
pred = bag.predict(X_train_aug)
pred_t = bag.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("voter score (train): %f, (test): %f"%(score_train, score_test))
candidates.append({'clf': bag, 'C':1.0, 'score':0.561196})
"""

LR score (train): 0.559638, (test): 0.554444
LR score CV: 0.550012
LR score (train): 0.559997, (test): 0.561799


'\nbag = BaggingClassifier(base_estimator=lr, n_estimators=100, max_samples=0.5, max_features=0.5,\n                        oob_score=True, random_state=0, n_jobs=3)\n\nscore = cross_val_score(bag, X_train_aug, y_train, cv=10)\n\n\n# best case here\nprint("voter score (CV): %f"%(score.mean()))\nbag.fit(X_train_aug, y_train)\npred = bag.predict(X_train_aug)\npred_t = bag.predict(X_htest)\nscore_train = metrics.accuracy_score(y_train, pred)\nscore_test = metrics.accuracy_score(y_htest, pred_t)\n\n# best case here\nprint("voter score (train): %f, (test): %f"%(score_train, score_test))\ncandidates.append({\'clf\': bag, \'C\':1.0, \'score\':0.561196})\n'

In [90]:
topic_test = tf_vectorizer.transform(df_test['title'])
topic_test = lda.fit_transform(topic_test)


In [248]:
doc_test = tfdoc.transform(df_test['content'])
test_doc_x = ch2.transform(doc_test)
aut_test = tfaut.transform(df_test['author'])
test_aut_x = cha2.transform(aut_test)

In [293]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values
test_channel = pd.read_csv('test_channel.csv').values
test_polarity = pd.read_csv('test_polarity.csv').values
test_links = pd.read_csv('test_links.csv').values
test_keypop = pd.read_csv('test_keypop.csv').values
test_nwords = df_test['n_words'].values.reshape(-1,1)
test_nfig = df_test['n_fig'].values.reshape(-1,1)
test_nword = df_test['n_words'].reshape(-1,1)
test_ner = pd.read_csv('test_ner.csv').values[:, 6:]


time_feats = np.concatenate((test_time, test_weekday), axis=1)
cont_feats = np.concatenate((test_polarity, topic_test, test_channel), axis=1)
stdn_feats = np.concatenate((n_twords_test, test_nwords, test_links, test_keypop, test_ner), axis=1)
stdn_feats = sc.transform(stdn_feats)

cont_feats = poly_cont.transform(cont_feats)[:,1:]
stdn_feats = poly_stdn.transform(stdn_feats)[:,1:]

X_test_con = np.concatenate((time_feats, cont_feats, stdn_feats, test_doc_x.todense()), axis=1)
#X_test_con = sc.transform(X_test_con)

/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [309]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test___%f.csv' % (cand['C']), index=False)